In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:/yogesh/data_sets/socialmedia-disaster-tweets-DFE.csv" , encoding='latin-1')

In [3]:
df = df.drop(['choose_one_gold','_unit_id','userid','_last_judgment_at','tweetid'] ,1)

In [4]:
df.head()

,_golden,_unit_state,_trusted_judgments,choose_one,choose_one:confidence,keyword,location,text
0,True,golden,156,Relevant,1.0000,NaN,NaN,Just happened a terrible car crash
1,True,golden,152,Relevant,1.0000,NaN,NaN,Our Deeds are the Reason of this #earthquake M...
2,True,golden,137,Relevant,1.0000,NaN,NaN,"Heard about #earthquake is different cities, s..."
3,True,golden,136,Relevant,0.9603,NaN,NaN,"there is a forest fire at spot pond, geese are..."
4,True,golden,138,Relevant,1.0000,NaN,NaN,Forest fire near La Ronge Sask. Canada


In [5]:
text_df = df[['text','choose_one']]

In [6]:
text_df.choose_one.value_counts() # we can apply smote

Not Relevant    6187
Relevant        4673
Can't Decide      16
Name: choose_one, dtype: int64

In [7]:
#standardization
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

text_df = standardize_text(text_df, "text")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [8]:
from nltk.corpus import stopwords
from string import punctuation

mystopwords = list(set(stopwords.words('english')).union(punctuation))

In [9]:
text_df.head()

,text,choose_one
0,just happened a terrible car crash,Relevant
1,our deeds are the reason of this earthquake m...,Relevant
2,"heard about earthquake is different cities, s...",Relevant
3,"there is a forest fire at spot pond, geese are...",Relevant
4,forest fire near la ronge sask canada,Relevant


In [10]:
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer , PorterStemmer
lan_stem = LancasterStemmer()
por_stem = PorterStemmer()

In [11]:
#stopwards removal and 
sents = text_df['text'].tolist()
for x in range(len(sents)):
    sents[x] = word_tokenize(sents[x])

clean_sents = []
for x in sents:
    clean_sent = ''
    for word in x:
        if word not in mystopwords:
            stem_word = lan_stem.stem(word)
            clean_sent = clean_sent +" "+ stem_word
    clean_sents.append(clean_sent.strip())

In [12]:
text_df['text'] = pd.Series(clean_sents)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
text_df['label_choose_one'] = le.fit_transform(text_df['choose_one'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [15]:
vectors = count_vect.fit_transform(text_df['text'])

In [16]:
arr_vect = vectors.toarray()

In [17]:
vect_df = pd.DataFrame(arr_vect)

In [18]:
text_df = pd.concat([vect_df,text_df] , axis=1)

In [19]:
vect_df.shape

(10876, 12747)

In [20]:
text_df.shape

(10876, 12750)

In [21]:
text_df = text_df.drop(['text','choose_one'] , 1)

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
log_reg = LogisticRegression()

In [24]:
from sklearn.model_selection import train_test_split as tts
X = text_df.drop(['label_choose_one'],1)
y = text_df['label_choose_one']
X_train,X_test,y_train,y_test = tts(X,y,test_size = 0.2,random_state=42)

In [25]:
log_reg.fit(X_train,y_train)

C:\Users\chyo8003\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chyo8003\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [26]:
y_pred = log_reg.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score ,confusion_matrix,classification_report

In [28]:
accuracy_score(y_pred,y_test)

0.8010110294117647

In [29]:
confusion_matrix(y_pred,y_test)

array([[   0,    0,    0],
       [   3, 1057,  271],
       [   0,  159,  686]], dtype=int64)

In [30]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.87      0.79      0.83      1331
           2       0.72      0.81      0.76       845

   micro avg       0.80      0.80      0.80      2176
   macro avg       0.53      0.54      0.53      2176
weighted avg       0.81      0.80      0.80      2176



C:\Users\chyo8003\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [31]:
df.isnull().sum()/df.shape[0]

_golden                  0.000000
_unit_state              0.000000
_trusted_judgments       0.000000
choose_one               0.000000
choose_one:confidence    0.000000
keyword                  0.007999
location                 0.334498
text                     0.000000
dtype: float64

In [32]:
df['keyword'].fillna(df['keyword'].mode()[0],inplace=True)

In [33]:
df.isnull().sum()/df.shape[0]

_golden                  0.000000
_unit_state              0.000000
_trusted_judgments       0.000000
choose_one               0.000000
choose_one:confidence    0.000000
keyword                  0.000000
location                 0.334498
text                     0.000000
dtype: float64

In [34]:
df['location'].fillna(df['location'].mode()[0],inplace=True)

In [35]:
df.isnull().sum()/df.shape[0]

_golden                  0.0
_unit_state              0.0
_trusted_judgments       0.0
choose_one               0.0
choose_one:confidence    0.0
keyword                  0.0
location                 0.0
text                     0.0
dtype: float64

In [36]:
df = df.drop(['choose_one'],1)

In [37]:
cat_cols = df.select_dtypes('object').columns.values.tolist()

In [38]:
for x in cat_cols:
    df[x] = le.fit_transform(df[x])

In [39]:
df['_golden'] = le.fit_transform(df['_golden'])

In [40]:
ml_df = pd.concat([df,text_df],axis = 1)

In [41]:
X = ml_df.drop(['label_choose_one'],1)
y = ml_df['label_choose_one']
X_train,X_test,y_train,y_test = tts(X,y,test_size = 0.2,random_state=42)

In [42]:
log_reg.fit(X_train,y_train)

C:\Users\chyo8003\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chyo8003\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [43]:
y_pred = log_reg.predict(X_test)

In [44]:
accuracy_score(y_test,y_pred)

0.8033088235294118

In [45]:
from sklearn.neighbors import KNeighborsClassifier

In [50]:
knn = KNeighborsClassifier(n_neighbors=10)

In [51]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [52]:
y_knn_pred = knn.predict(X_test)

In [53]:
accuracy_score(y_test,y_knn_pred)

0.5919117647058824

In [54]:
from sklearn.ensemble import RandomForestClassifier

In [55]:
rfc = RandomForestClassifier()

In [56]:
rfc.fit(X_train,y_train)

C:\Users\chyo8003\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [57]:
y_rfc_pred = rfc.predict(X_test)

In [58]:
accuracy_score(y_test,y_rfc_pred)

0.7858455882352942

In [59]:
from sklearn.tree import DecisionTreeClassifier

In [60]:
dtc = DecisionTreeClassifier()

In [62]:
dtc.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [63]:
y_dtc_pred = dtc.predict(X_test)

In [64]:
accuracy_score(y_test,y_dtc_pred)

0.7587316176470589

In [71]:
from sklearn.naive_bayes import GaussianNB , BernoulliNB

In [66]:
nbg = GaussianNB()

In [67]:
nbg.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [69]:
y_nbg_pred = nbg.predict(X_test)

In [70]:
accuracy_score(y_test,y_nbg_pred)

0.09053308823529412

In [72]:
nbb = BernoulliNB()

In [73]:
nbb.fit(X_train,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [74]:
y_nbb_pred = nbb.predict(X_test)

In [75]:
accuracy_score(y_test,y_nbb_pred)

0.7950367647058824

In [76]:
from sklearn.model_selection import GridSearchCV

In [83]:
params = {"C":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,3,4,5,6,7,8,9,10]}
gscv = GridSearchCV(log_reg,params,cv=5)

In [84]:
import warnings
warnings.filterwarnings('ignore')
gscv.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [85]:
gscv.best_params_

{'C': 0.4}

In [86]:
log_reg  =LogisticRegression(C=0.4)

In [87]:
log_reg.fit(X_train,y_train)

LogisticRegression(C=0.4, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [88]:
y_log_pred = log_reg.predict(X_test)

In [89]:
accuracy_score(y_test,y_log_pred)

0.8028492647058824

In [90]:
confusion_matrix(y_test,y_log_pred)

array([[   0,    3,    0],
       [   0, 1056,  160],
       [   0,  266,  691]], dtype=int64)

In [91]:
log_reg  =LogisticRegression(C=1)

In [92]:
log_reg.fit(X_train,y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [93]:
y_pred_log_1 = log_reg.predict(X_test)

In [94]:
accuracy_score(y_test,y_pred_log_1)

0.8033088235294118

In [95]:
from imblearn.over_sampling import SMOTE

In [96]:
smote = SMOTE(random_state=42)

In [98]:
X = ml_df.drop(['label_choose_one'],1)
y = ml_df['label_choose_one']
X_res , y_res = smote.fit_sample(X,y)

In [102]:
y_smote = pd.Series(y_res)

In [105]:
X_smote = pd.DataFrame(X_res)

In [106]:
X_train,X_test,y_train,y_test = tts(X_smote,y_smote,test_size = 0.2,random_state=42)

In [107]:
log_reg.fit(X_train,y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [108]:
y_smote = log_reg.predict(X_test)

In [109]:
accuracy_score(y_test,y_smote)

0.8685698895771613

In [110]:
confusion_matrix(y_test,y_smote)

array([[1221,    0,    0],
       [   6, 1064,  228],
       [  12,  242,  940]], dtype=int64)

In [111]:
print(classification_report(y_test,y_smote))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1221
           1       0.81      0.82      0.82      1298
           2       0.80      0.79      0.80      1194

   micro avg       0.87      0.87      0.87      3713
   macro avg       0.87      0.87      0.87      3713
weighted avg       0.87      0.87      0.87      3713

